In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.optimizers import adam_v2
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

import cv2
from cv2 import imread, createCLAHE # read and equalize images

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
from skimage.util import montage as montage2d

import os
from glob import glob
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
!mkdir /home/TBX11K/BF

!mkdir /home/TBX11K/BF/train
!mkdir /home/TBX11K/BF/val
!mkdir /home/TBX11K/BF/test

!mkdir /home/TBX11K/BF/train/sick
!mkdir /home/TBX11K/BF/train/health
!mkdir /home/TBX11K/BF/train/tb

!mkdir /home/TBX11K/BF/val/sick
!mkdir /home/TBX11K/BF/val/health
!mkdir /home/TBX11K/BF/val/tb


!mkdir /home/TBX11K/BF/test/Montgomery
!mkdir /home/TBX11K/BF/test/Montgomery/health
!mkdir /home/TBX11K/BF/test/Montgomery/tb
!mkdir /home/TBX11K/BF/test/Montgomery/sick

!mkdir /home/TBX11K/BF/test/Shenzhen
!mkdir /home/TBX11K/BF/test/Shenzhen/health
!mkdir /home/TBX11K/BF/test/Shenzhen/tb
!mkdir /home/TBX11K/BF/test/Shenzhen/sick

In [2]:
!mkdir /home/TBX11K/BF/test/DA_DB
!mkdir /home/TBX11K/BF/test/DA_DB/health
!mkdir /home/TBX11K/BF/test/DA_DB/tb
!mkdir /home/TBX11K/BF/test/DA_DB/sick

In [3]:
from multiprocessing import Pool

def process_clahe(res):
    filename = res['filename']
    dest_file = res['dest_file']
    
    image = cv2.imread(filename,cv2.IMREAD_GRAYSCALE)
    bf_image = cv2.bilateralFilter(image, 9, 80, 80)
    cv2.imwrite(dest_file,bf_image)
    return True

def process_image_files(img_files,dest_dir):
    res = []
    for filename in img_files:
        dest_file = os.path.join(dest_dir,os.path.basename(filename))
        row = {'filename':filename,'dest_file':dest_file}
        res.append(row)
    
    workers = os.cpu_count()
    with Pool(workers) as p:
        p.map(process_clahe, res)  

In [6]:
TBX_DIR = '/home/TBX11K'
TBX_DEST_DIR = '/home/TBX11K/BF'
for split_dir in ['train','val']:
    for class_dir in ['health','sick','tb']:
        file_path = os.path.join(TBX_DIR,split_dir,class_dir,'*.png')
        img_files = glob(file_path)
        dest_dir = os.path.join(TBX_DEST_DIR,split_dir,class_dir)
        #dest_file = os.path.join(dest_dir,os.path.basename(img_file))
        #process_clahe({'filename':img_file,'dest_file':dest_file})
        process_image_files(img_files,dest_dir)
        

In [7]:
TBX_DIR_TEST = '/home/TBX11K/test'
TBX_DEST_DIR_TEST = '/home/TBX11K/BF/test'
for split_dir in ['Montgomery','Shenzhen']:
    for class_dir in ['health','sick','tb']:
        file_path = os.path.join(TBX_DIR_TEST,split_dir,class_dir,'*.png')
        img_files = glob(file_path)
        dest_dir = os.path.join(TBX_DEST_DIR_TEST,split_dir,class_dir)
        
        #dest_file = os.path.join(dest_dir,os.path.basename(img_files[0]))
        #print(img_files[0],dest_file)
        #break
        #process_clahe({'filename':img_file,'dest_file':dest_file})
        process_image_files(img_files,dest_dir)

In [4]:
TBX_DIR_TEST = '/home/TBX11K/test'
TBX_DEST_DIR_TEST = '/home/TBX11K/BF/test'
for split_dir in ['DA_DB']:
    for class_dir in ['tb']:
        file_path = os.path.join(TBX_DIR_TEST,split_dir,class_dir,'*.png')
        img_files = glob(file_path)
        dest_dir = os.path.join(TBX_DEST_DIR_TEST,split_dir,class_dir)
        
        #dest_file = os.path.join(dest_dir,os.path.basename(img_files[0]))
        #print(img_files[0],dest_file)
        #break
        #process_clahe({'filename':img_file,'dest_file':dest_file})
        process_image_files(img_files,dest_dir)